In [20]:
import numpy as np
import xroms
from glob import glob
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd
import xrft as xrft
import warnings
import sys
import xrft
sys.path.append("/homes/metogra/iufarias/FeedbackSubmeso/useful/")
import romspickle

from dask.diagnostics import ProgressBar
import dask
from dask.distributed import Client, LocalCluster

import scipy.integrate as integ
warnings.filterwarnings("ignore")
import seawater as sw


In [21]:
client = Client(processes=False)
# client = Client(cluster)
# client = Client(n_workers=90)


In [22]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://128.8.216.13:38735/status,
Dashboard: http://128.8.216.13:38735/status,Workers: 1
Total threads: 128,Total memory: 502.98 GiB
Status: running,Using processes: False
Comm: inproc://128.8.216.13/521631/11,Workers: 1
Dashboard: http://128.8.216.13:38735/status,Total threads: 128
Started: Just now,Total memory: 502.98 GiB
Comm: inproc://128.8.216.13/521631/14,Total threads: 128
Dashboard: http://128.8.216.13:43917/status,Memory: 502.98 GiB
Nanny: None,


In [24]:
# chunks={'time_counter':10, 'x_rho':50, 'y_rho':190,'s_rho':80,'s_w':80}
chunks={'time_counter':1,'x_rho':1000,"x_u":1000,"x_v":1000,
                        'y_rho':1520,"y_u":1520,"y_v":1520,'s_rho':80,'s_w':80}

M='03'
path='/data/pacific/lrenault/SASI/CROCO/'

varf=xr.open_mfdataset(path + 'FULL/SASI*2012-{0}*.nc'.format(M),data_vars='minimal',chunks=chunks)
vart=xr.open_mfdataset(path + 'SMTH/SASI*2012-{0}*.nc'.format(M),data_vars='minimal',chunks=chunks)

# varf=xr.open_mfdataset(path + 'FULL/SASI*2012*.nc',data_vars='minimal',chunks=chunks)
# vart=xr.open_mfdataset(path + 'SMTH/SASI*2012*.nc',data_vars='minimal',chunks=chunks)

In [25]:
varx = varf.rename({"time_counter":"ocean_time","x_rho": "xi_rho","x_u":"xi_u","x_v":"xi_v","x_w":"xi_w",
                    "y_rho":"eta_rho","y_v":"eta_v","y_u":"eta_u","y_w":"eta_w","s_rho":"s_rho","s_w":"s_w"})
df,gf=xroms.roms_dataset(varx,Vtransform=varx.Vtransform.data)

vary = vart.rename({"time_counter":"ocean_time","x_rho": "xi_rho","x_u":"xi_u","x_v":"xi_v",
                    "y_rho":"eta_rho","y_v":"eta_v","y_u":"eta_u","y_w":"eta_w","s_rho":"s_rho","s_w":"s_w"})
dt,gt=xroms.roms_dataset(vary,Vtransform=vary.Vtransform.data)

In [26]:
timer_h=pd.DatetimeIndex(dt.ocean_time)
timer_d=pd.DatetimeIndex(dt.ocean_time.groupby("ocean_time.dayofyear").mean())

In [27]:
# temp_full=df.temp[:,:,120:-120,120:700]
# temp_smth=dt.temp[:,:,120:-120,120:700]

# salt_full=df.salt[:,:,120:-120,120:700]
# salt_smth=dt.salt[:,:,120:-120,120:700]

In [28]:
# def xr_pden(s,t,p,pr):
#     return xr.apply_ufunc(sw.pden,kwargs={'t':t,'s':s,'p':p,'pr':pr})

In [29]:
%time
rho_full=df.rho[:,:,120:-120,120:700]+df.rho0
rho_smth=dt.rho[:,:,120:-120,120:700]+dt.rho0

# rho_full=temp_full[0].copy(data=sw.pden(s=salt_full[0,:],t=temp_full[0,:],p=-temp_full.z_rho[0,:],pr=0))
# rho_smth=temp_smth[0].copy(data=sw.pden(s=salt_smth[0,:],t=temp_smth[0,:],p=-temp_smth.z_rho[0,:],pr=0))

CPU times: user 11 µs, sys: 5 µs, total: 16 µs
Wall time: 31.9 µs


In [30]:
# z_interp=np.linspace(-2,-1000, 100)
z_interp=np.linspace(-2,-1000, 200)

# z_interp=np.linspace(-2,-1000,100)
# z_interp_rev=np.linspace(2,1000,100)[::-1]

np.diff(z_interp).mean()

-5.015075376884422

In [31]:
# z_matrix=np.zeros([z_interp.shape[0],rho_full.shape[2],rho_full.shape[3]])
z_matrix=np.zeros([z_interp.shape[0],rho_full.shape[1],rho_full.shape[2]])
# z_matrix[:,:,:]=-z_interp[:,None,None]

In [32]:
dec=1
Zstar_full=xr.DataArray(coords=(df.ocean_time[::dec],
                                z_interp,
                                rho_full.eta_rho.data,
                                rho_full.xi_rho.data),
                      dims=['ocean_time','z_rho','eta_rho','xi_rho'])

Rho_full_interp=xr.DataArray(coords=(df.ocean_time[::dec],
                                z_interp,
                                rho_full.eta_rho.data,
                                rho_full.xi_rho.data),
                      dims=['ocean_time','z_rho','eta_rho','xi_rho'])

In [ ]:
%%time
"NEW METHOD"


for day_x in range(rho_full.ocean_time.shape[0]):


    rho_full_interp=rho_full[day_x,:,:,:].xroms.isoslice(z_interp).compute()
    # rho_full_interp['z_rho']=z_interp
    Rho_full_interp[day_x,:,:,:]=rho_full_interp

    "numpy method (working but janky)"
    rho_full_stacked=np.ravel(rho_full_interp)
    index_sorted_full=np.argsort(rho_full_stacked)

    rho_star=rho_full_interp.copy(data=rho_full_stacked[index_sorted_full].reshape(rho_full_interp.shape))
    rho_star_m=rho_full_interp.copy(data=np.tile(rho_star.mean(['xi_rho','eta_rho']),
                                                 (rho_full_interp.shape[2],rho_full_interp.shape[1],1)).T)

    for zi in range(z_interp.shape[0]):
        Zstar_full[day_x,zi,:,:]=z_interp[(np.abs(rho_full_interp[zi,:,:]-rho_star_m)).argmin('z_rho')]

In [ ]:
%%time
Zstar_full.to_netcdf('/data/pacific/iufarias/z_star/z_star_full_newmethod_M{0}.nc'.format(M))
del Zstar_full

In [ ]:
%%time
Rho_full_interp.to_netcdf('/data/pacific/iufarias/z_star/rho_full_M{0}.nc'.format(M))
del Rho_full_interp

In [ ]:
Zstar_smth=xr.DataArray(coords=(dt.ocean_time[::dec],
                                z_interp,
                                rho_smth.eta_rho,
                                rho_smth.xi_rho),
                      dims=['ocean_time','z_rho','eta_rho','xi_rho'])

Rho_smth_interp=xr.DataArray(coords=(dt.ocean_time[::dec],
                                z_interp,
                                rho_smth.eta_rho,
                                rho_smth.xi_rho),
                      dims=['ocean_time','z_rho','eta_rho','xi_rho'])

In [ ]:
%%time
"NEW METHOD"
for day_x in range(rho_smth.ocean_time.shape[0]):


    rho_smth_interp=rho_smth[day_x,:,:,:].xroms.isoslice(z_interp).compute()
    # rho_smth_interp['z_rho']=z_interp_rev
    Rho_smth_interp[day_x,:,:,:]=rho_smth_interp

    "numpy working hod (working but janky)"
    rho_smth_stacked=np.ravel(rho_smth_interp)
    index_sorted_smth=np.argsort(rho_smth_stacked)

    rho_star=rho_smth_interp.copy(data=rho_smth_stacked[index_sorted_smth].reshape(rho_smth_interp.shape))
    rho_star_m=rho_smth_interp.copy(data=np.tile(rho_star.mean(['xi_rho','eta_rho']),
                                                 (rho_smth_interp.shape[2],rho_smth_interp.shape[1],1)).T)

    for zi in range(z_interp.shape[0]):
        Zstar_smth[day_x,zi,:,:]=z_interp[(np.abs(rho_smth_interp[zi,:,:]-rho_star_m)).argmin('z_rho')]

In [ ]:
%%time
Zstar_smth.to_netcdf('/data/pacific/iufarias/z_star/z_star_smth_newmethod_M{0}.nc'.format(M))
del Zstar_smth

In [ ]:
%%time
Rho_smth_interp.to_netcdf('/data/pacific/iufarias/z_star/rho_smth_M{0}.nc'.format(M))
del Rho_smth_interp